In [1]:
import seaborn as sns
sns.set()

In [2]:
from static_grader import grader

# Spark Miniproject


Stack Overflow is a collaboratively edited question-and-answer site originally focused on programming topics. Because of the variety of features tracked, including a variety of feedback metrics, it allows for some open-ended analysis of user behavior on the site.

Stack Exchange (the parent organization) provides an anonymized [data dump](https://archive.org/details/stackexchange), and we'll use Spark to perform data manipulation, analysis, and machine learning on this data set. As a side note, there's also an online data explorer which allows you to query the data interactively.

*Consider*: Do we need to use Spark to work with this data set? What are our alternatives?

## Workflow


**All questions in this miniproject can be done locally in this notebook (i.e. on your Jupyter pod).**  

You are free to try running on a cloud service, but note that we have no resources to pay for you to try out these services.  (New users often get a limited amount of free credit to try a service.)  Also, the grader library will not be available, so you would have to get your answers into this notebook to submit to the grader.   See the appropriate lecture notebooks for information on how to use cloud services if you want to try them out.

Python example workflow when **not** running in a Jupyter notebook:

1. Edit source code in your `main.py` file, classes in a separate `classes.py` (class definitions need to be written in a separate file and then included at runtime).
1. If you are using a cloud service, in order to make your code more flexible, it's recommended to incorporate command-line arguments that specify the location of the input data and where output should be written.
``` python
# Command line arguments using sysv or argparse in Python
if __name__ == '__main__':
    main(ARGS.input_dir, ARGS.output_dir)
```
1. Run locally using the `spark-submit` program on a chunk using, eg., `$SPARK_HOME/bin/spark-submit --py-files src/classes.py src/main.py data/stats results/stats/`  Note that long jobs using `spark-submit` may not finish before your server gets automatically shut down (our server only checks for running Jupyter notebooks to avoid shutting down).  
1. Run on Amazon Web Services (AWS) once your testing and development are done.  Note that you will also have to load all of the input data on an AWS bucket.  (Similar statements apply if you were to use Google Cloud Platform (GCP) or other services.)  

General tips when using `spark-submit` or working on a cloud service:
* Try `cat output_dir/* | sort -n -t , -k 1.2 -o sorted_output` to concatenate your output files, which will be in `part-xxxxx` format.
* You can alternatively access an interactive PySpark shell on your Jupyter pod with this command: `$SPARK_HOME/bin/pyspark`

## Accessing the data


The data is available on S3 (`s3://dataincubator-course/spark-stack-data`). There are three sub-folders, `allUsers`, `allPosts`, and `allVotes` which contain Gzipped XML.  The `allPosts` sub-folder will contain data with the following format:

``` html
<row Body="&lt;p&gt;I always validate my web pages, and I recommend you do the same BUT many large company websites DO NOT and cannot validate because the importance of the website looking exactly the same on all systems requires rules to be broken. &lt;/p&gt;&#10;&#10;&lt;p&gt;In general, valid websites help your page look good even on odd configurations (like cell phones) so you should always at least try to make it validate.&lt;/p&gt;&#10;" CommentCount="0" CreationDate="2008-10-12T20:26:29.397" Id="195995" LastActivityDate="2008-10-12T20:26:29.397" OwnerDisplayName="Eric Wendelin" OwnerUserId="25066" ParentId="195973" PostTypeId="2" Score="0" />
```

Data from the much smaller `stats.stackexchange.com` website (called "Cross Validated") is available in the same format on S3 (`s3://dataincubator-course/spark-stats-data`). This smaller data set will be used below in most questions to avoid working with the full data set for every question.

The full schema is available as a text file, which can be downloaded with the following command.

In [3]:
!aws s3 cp s3://dataincubator-course/spark-stats-data/stack_exchange_schema.txt .

download: s3://dataincubator-course/spark-stats-data/stack_exchange_schema.txt to ./stack_exchange_schema.txt


You can either get the data by running the appropriate S3 commands in the terminal, or by running this block for the smaller stats data set:

In [4]:
!mkdir -p spark-stats-data
!aws s3 sync --exclude '*' --include 'all*' s3://dataincubator-course/spark-stats-data/ ./spark-stats-data
!aws s3 sync --exclude '*' --include 'posts*zip' s3://dataincubator-course/spark-stats-data/ ./spark-stats-data

And to get the much larger full data set (be warned, this can take 20 or more minutes, so you may want to run it in the terminal to avoid locking up the notebook):

In [5]:
!mkdir -p spark-stack-data
!aws s3 sync --exclude '*' --include 'all*' s3://dataincubator-course/spark-stack-data/ ./spark-stack-data

## Data input and parsing


Some rows are split across multiple lines; these can be discarded. Incorrectly formatted XML can also be ignored. It is enough to simply skip problematic rows, the loss of data will not significantly impact our results on these large data sets.

**You will need to handle XML parsing yourself.  Our solution uses `lxml.etree` in Python, and we would recommend using this tool yourself to handle the XML parsing.**

The goal should be to have a parsing function that can be applied to the input data to access any desired XML elements. You might find it convenient to represent the post, votes, users, etc. data using [`namedtuples`](https://docs.python.org/3/library/collections.html?highlight=namedtuple#collections.namedtuple).

## Structure

This miniproject is divided into two parts, called `spark_data` and `spark_ml`. The first part is doing data analysis in spark, on both a small data set and a large one. This consists of the first six questions in the notebook. The second part is using Spark ML to do machine learning, and is the last two questions. They are distinguished both by sections in the notebook and the question names.

## Spark data section

## Question 1: Bad XML


This first question is a simple question to test your parsing code. Create an RDD of Post objects where each Post is a valid row of XML from the small "Cross Validated" (stats.stackexchange.com) `allPosts` data set.

We are going to take several shortcuts to speed up and simplify our computations.  First, your parsing function should only attempt to parse rows that start with `<row` as these denote actual data entries. This should be done in Spark as the data is being read in from disk, without any pre-Spark processing. 

Return the total number of XML rows that started with `<row` that were subsequently **rejected** during your XML processing.  Note that the text is Unicode, and contains non-ASCII characters.  You may need to re-encode to UTF-8 (depending on your XML parser).

**Note that this cleaned data set will be used for questions 1-5.**  (For questions 6-8, you want to similarly remove improperly formatted XML from that data before proceeding further.)  

*Question*: Can you figure out what filters you need to put in place to avoid throwing parsing errors entirely?

In [3]:
from pyspark import SparkContext
from lxml import etree 

sc = SparkContext.getOrCreate()

def parseLines(xml_string):
    parser = etree.XMLParser()
    try:
        xml_root = etree.fromstring(xml_string, parser)
    except Exception:
        return False
    return True
    

lines = (sc.textFile('spark-stats-data/allPosts/')
           .filter(lambda txt: len(txt.split())>0 and len(txt.split()[0])>=4 and txt.split()[0][:4]=='<row'))

good_lines= (sc.textFile('spark-stats-data/allPosts/')
               .filter(lambda txt: len(txt.split())>0 and len(txt.split()[0])>=4 and txt.split()[0][:4]=='<row')
               .filter(parseLines))


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
bad_xml = lines.count() - good_lines.count()

grader.score('spark_data__bad_xml', bad_xml)

Your score: 1.0000


## Question 2: Favorites and scores

We're interested in looking for useful patterns in the data.  If we look at the Post data again (the smaller set, `stats.stackexchange.com`), we see that many things about each post are recorded.  We're going to start by looking to see if there is a relationship between the number of times a post was favorited (the `FavoriteCount`) and the `Score`.  The score is the number of times the post was upvoted minus the number of times it was downvoted, so it is a measure of how much a post was liked.  We'd expect posts with a higher number of favorites to have better scores, since they're both measurements of how good the post is.

Let's aggregate posts by the number of favorites, and find the average score for each number of favorites.  Do this for the lowest 50 numbers of favorites.

**If any field in the Posts or Users is missing, such as the `FavoriteCount`, you should assume it is zero. _Make this assumption for all questions going forward._**

_Note:_ Before submitting, take a look at the numbers.  Do they follow the trend you expect?

**Checkpoints**

- Total score across all posts: 299469
- Mean of first 50 favorite counts (averaging the keys themselves): 24.76

In [8]:
from collections import namedtuple
record_tuple = namedtuple('record_tuple', ['postId','FavoriteCount','Score', 'postType'])


def parsePost(xml_string):
    parser = etree.XMLParser()
    xml_root = etree.fromstring(xml_string, parser)
    Map=xml_root.attrib
    postId=int(Map['Id'])
    if 'FavoriteCount' in Map:
        FavoriteCount=int(Map['FavoriteCount'])
    else:
        FavoriteCount=0
    if 'Score' in Map:
        Score=int(Map['Score'])
    else:
        Score=0
    postType=int(Map['PostTypeId'])
    return record_tuple(postId, FavoriteCount, Score, postType)
       
# xml_list=good_lines.take(10)
# for xml_string in xml_list:
#     print(parsePost(xml_string))

parsedData=(good_lines.map(parsePost)
                     .map(lambda record: (record.FavoriteCount, (record.Score, 1)))
                     .reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1])))
favs=parsedData.map(lambda c: (c[0], -1 if c[1][1]==0 else c[1][0]/c[1][1])).distinct()

allData=favs.take(10000)
allData.sort()


In [9]:
favorite_score = allData[:50]

grader.score('spark_data__favorite_score', favorite_score)

Your score: 1.0000


## Question 3: Answer percentage


Investigate the correlation between a user's reputation and the kind of posts they make. For the 99 users with the highest reputation, single out posts which are either questions or answers and look at the percentage of these posts that are answers: *(answers / (answers + questions))*. 

Return a tuple of their **user ID** and this fraction.

You should also return (-1, fraction) to represent the case where you average over all users (so you will return 100 entries total).

Again, you only need to run this on the statistics overflow set.


#### Checkpoints

* Total questions: 52,060
* Total answers: 55,304
* Top 99 users' average reputation: 11893.464646464647

In [10]:
user_lines = (sc.textFile('spark-stats-data/allUsers/')
           .filter(lambda txt: len(txt.split())>0 and len(txt.split()[0])>=4 and txt.split()[0][:4]=='<row'))
user_lines.take(4)

def parseUserLines(xml_string):
    parser = etree.XMLParser()
    try:
        xml_root = etree.fromstring(xml_string, parser)
        Map=xml_root.attrib
        userId=int(Map['Id'])        
    except Exception:
        return False
    return True

record_tuple = namedtuple('record_tuple', ['userId','Reputation'])

def parseUser(xml_string):
    parser = etree.XMLParser()
    xml_root = etree.fromstring(xml_string, parser)
    Map=xml_root.attrib
    userId=int(Map['Id'])
    Reputation=0 if 'Reputation' not in Map else int(Map['Reputation'])
    return record_tuple(userId, Reputation)

good_user_lines= (user_lines.filter(parseUserLines)
                            .map(parseUser))

user_summary=(good_user_lines.map(lambda record: (-record.Reputation, record.userId))
                             .sortByKey())

top_users=[i[1] for i in user_summary.take(99)]

In [11]:
def FilterUser(xml_string):
    parser = etree.XMLParser()
    xml_root = etree.fromstring(xml_string, parser)
    Map=xml_root.attrib
    return 'OwnerUserId' in Map

record_tuple = namedtuple('record_tuple', ['userId','postType'])

def parseUser(xml_string):
    parser = etree.XMLParser()
    xml_root = etree.fromstring(xml_string, parser)
    Map=xml_root.attrib
    userId=int(Map['OwnerUserId'])
    postType=int(Map['PostTypeId'])
    return record_tuple(userId, postType)

userData=(good_lines.filter(FilterUser)
                    .map(parseUser)
                    .map(lambda record: (record.userId, (1 if record.postType!=1 else 0, 1)))
                    .reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1])))

QnA_summary=(userData.distinct()
                     .map(lambda c: (c[0], c[1][0]/c[1][1])))



tot_summary=(userData.map(lambda c: (1, (c[1][0], c[1][1])))
                     .reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1]))
                     .distinct()
                     .map(lambda c: (c[0], c[1][0]/c[1][1]))).take(1)

QnA_summary=QnA_summary.take(QnA_summary.count())

from collections import defaultdict
frac_map=defaultdict(int)
for user, fraction in QnA_summary:
    frac_map[user]=fraction


answer_percentage=[(-1, tot_summary[0][1])]+[(user, frac_map[user]) for user in top_users]

In [12]:
#answer_percentage = [(7071, 0.9107142857142857)] * 100

grader.score('spark_data__answer_percentage', answer_percentage)

Your score: 1.0000


## Question 4: First question

We'd expect the first **question** a user asks to be indicative of their future behavior.  We'll dig more into that in the next problem, but for now let's see the relationship between reputation and how long it took each person to ask their first question.

For each user that asked a question, find the difference between when their account was created (`CreationDate` for the User) and when they asked their first question (`CreationDate` for their first question).  Return this time difference in days (round down, so 2.7 days counts as 2 days) for the 100 users with the highest reputation, in the form

`(UserId, Days)`

**Checkpoints**
- Users that asked a question: 23134
- Average number of days (round each user's days, then average): 30.1074258

In [13]:
def parseUserLines(xml_string):
    parser = etree.XMLParser()
    try:
        xml_root = etree.fromstring(xml_string, parser)
        Map=xml_root.attrib
        userId=int(Map['Id'])        
    except Exception:
        return False
    return True

user_tuple = namedtuple('user_tuple', ['userId','Reputation', 'CreationDateUser'])

def parseUser(xml_string):
    parser = etree.XMLParser()
    xml_root = etree.fromstring(xml_string, parser)
    Map=xml_root.attrib
    userId=int(Map['Id'])
    Reputation=0 if 'Reputation' not in Map else int(Map['Reputation'])
    return user_tuple(userId, Reputation, Map['CreationDate'])

userData= (user_lines.filter(parseUserLines)
                            .map(parseUser))

In [14]:
def FilterPost(xml_string):
    parser = etree.XMLParser()
    xml_root = etree.fromstring(xml_string, parser)
    Map=xml_root.attrib
    return 'OwnerUserId' in Map

post_tuple = namedtuple('post_tuple', ['userId','CreationDatePost', 'isQuestion'])

def parsePost(xml_string):
    parser = etree.XMLParser()
    xml_root = etree.fromstring(xml_string, parser)
    Map=xml_root.attrib
    userId=int(Map['OwnerUserId'])
    return post_tuple(userId, Map['CreationDate'], int(Map['PostTypeId'])==1)

postData=(good_lines.filter(FilterPost)
                    .map(parsePost)
                    .filter(lambda record: record.isQuestion))


In [15]:
min_rep=-(userData.map(lambda record: (-record.Reputation, record.userId))
                 .sortByKey()).take(300)[-1][0]
userData=userData.filter(lambda record: record.Reputation>=min_rep)
userData.count()
            

300

In [16]:
info=userData.map(lambda c: (c.userId, c.Reputation)).collect()

repu_map={}
for ID, repu in info:
    repu_map[ID]=repu
    

In [17]:
from datetime import datetime, timedelta

UserPostInfo = (postData.map(lambda p: (p.userId, p.CreationDatePost)) 
                .join(userData.map(lambda u: (u.userId, u.CreationDateUser)))
                .reduceByKey(lambda a, b: (min(a[0], b[0]), min(a[1], b[1])))
                .map(lambda c: (c[0], int(timedelta.total_seconds(datetime.strptime(c[1][0], "%Y-%m-%dT%H:%M:%S.%f")-datetime.strptime(c[1][1], "%Y-%m-%dT%H:%M:%S.%f"))/86400)))).collect()

first_question_extra=[]
for item in UserPostInfo:
    first_question_extra.append([repu_map[item[0]], item[0], item[1]])

In [18]:
first_question=[(i[1], i[2]) for i in sorted(first_question_extra, reverse=True)][:100]

In [19]:
#first_question = [(805, 669)] * 100

grader.score('spark_data__first_question', first_question)

Your score: 1.0000


## Question 5: Identify veterans


It can be interesting to think about what factors influence a user to remain active on the site over a long period of time. In order not to bias the results towards older users, we'll define a time window between 100 and 150 days after account creation. If the user has made a post in this time, we'll consider them active and well on their way to being veterans of the site; if not, they are inactive and were likely brief users.

*Consider*: What other parameterizations of "activity" could we use, and how would they differ in terms of splitting our user base?

*Consider*: What other biases are still not dealt with, after using the above approach?

Let's see if there are differences between the first ever question posts of "veterans" vs. "brief users". For each group separately, average the score, views, number of answers, and number of favorites of the users' **first question**. Remember, if the score, views, answers, or favorites is missing, you should assume it is zero.

*Consider*: What story could you tell from these numbers? How do the numbers support it?


#### Checkpoints

* Total brief users: 24,864
* Total veteran users: 2,027

In [20]:
def FilterUser(xml_string):
    parser = etree.XMLParser()
    try:
        xml_root = etree.fromstring(xml_string, parser)
        Map=xml_root.attrib
        userId=int(Map['Id'])        
    except Exception:
        return False
    return True

user_tuple = namedtuple('user_tuple', ['userId', 'CreationDateUser'])

def parseUser(xml_string):
    parser = etree.XMLParser()
    xml_root = etree.fromstring(xml_string, parser)
    Map=xml_root.attrib
    userId=int(Map['Id'])
    return user_tuple(userId, Map['CreationDate'])

userData= (user_lines.filter(FilterUser)
                     .map(parseUser))


def FilterPost(xml_string):
    parser = etree.XMLParser()
    xml_root = etree.fromstring(xml_string, parser)
    Map=xml_root.attrib
    return 'OwnerUserId' in Map

post_tuple = namedtuple('post_tuple', ['userId','CreationDatePost', 'FavoriteCount', 'postType', 'AnswerCount', 'Score', 'ViewCount'])

def parsePost(xml_string):
    parser = etree.XMLParser()
    xml_root = etree.fromstring(xml_string, parser)
    Map=xml_root.attrib
    userId=int(Map['OwnerUserId'])
    if 'FavoriteCount' not in Map:
        FavoriteCount=0
    else:
        FavoriteCount=int(Map['FavoriteCount'])
    if 'Score' not in Map:
        Score=0
    else:
        Score=int(Map['Score']) 
    if 'ViewCount' not in Map:
        ViewCount=0
    else:
        ViewCount=int(Map['ViewCount'])   
    if 'AnswerCount' not in Map:
        AnswerCount=0
    else:
        AnswerCount=int(Map['AnswerCount'])         
    postType=int(Map['PostTypeId'])    
    return post_tuple(userId, Map['CreationDate'], FavoriteCount, postType, AnswerCount, Score, ViewCount)

postData=(good_lines.filter(FilterPost)
                    .map(parsePost))


In [21]:
def dateDiff(nxt, prev):
    prev_time=datetime.strptime(prev, "%Y-%m-%dT%H:%M:%S.%f")
    nxt_time=datetime.strptime(nxt, "%Y-%m-%dT%H:%M:%S.%f")
    return int(timedelta.total_seconds(nxt_time-prev_time)/86400)

freq_user_type=namedtuple('freq_user_type', ['userId','user_type'])

vetInfo = (postData.map(lambda p: (p.userId, p.CreationDatePost)) 
                        .join(userData.map(lambda u: (u.userId, u.CreationDateUser)))
                        .reduceByKey(lambda a, b: b if (dateDiff(b[0], b[1])>=100 and dateDiff(b[0], b[1])<=150) else a)
                        .map(lambda c: freq_user_type(c[0], 'vet' if (dateDiff(c[1][0], c[1][1])>=100 and dateDiff(c[1][0], c[1][1])<=150) else 'brief')))

vetInfo.take(20)

[freq_user_type(userId=36127, user_type='brief'),
 freq_user_type(userId=41197, user_type='brief'),
 freq_user_type(userId=41015, user_type='brief'),
 freq_user_type(userId=30589, user_type='brief'),
 freq_user_type(userId=41249, user_type='brief'),
 freq_user_type(userId=8333, user_type='vet'),
 freq_user_type(userId=10998, user_type='brief'),
 freq_user_type(userId=8580, user_type='brief'),
 freq_user_type(userId=17459, user_type='brief'),
 freq_user_type(userId=40976, user_type='vet'),
 freq_user_type(userId=41301, user_type='brief'),
 freq_user_type(userId=34658, user_type='vet'),
 freq_user_type(userId=38012, user_type='brief'),
 freq_user_type(userId=8515, user_type='brief'),
 freq_user_type(userId=26338, user_type='brief'),
 freq_user_type(userId=41392, user_type='brief'),
 freq_user_type(userId=30342, user_type='brief'),
 freq_user_type(userId=40378, user_type='brief'),
 freq_user_type(userId=11037, user_type='brief'),
 freq_user_type(userId=20657, user_type='brief')]

In [22]:
FavScoreSummary=(postData.filter(lambda record: record.postType==1)
                         .map(lambda p: (p.userId, (p.FavoriteCount, p.AnswerCount, p.Score, p.ViewCount, p.CreationDatePost)))
                         .join(vetInfo.map(lambda v: (v.userId, v.user_type)))
                         .map(lambda c: ((c[0], c[1][1]), c[1][0]))
                         .reduceByKey(lambda a, b: b if b[4]<a[4] else a)
                         .map(lambda c: (c[0][1], (c[1][0], c[1][1], c[1][2], c[1][3], 1)))
                         .reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1], a[2]+b[2], a[3]+b[3], a[4]+b[4]))).collect() 

identify_veterans={}
for item in FavScoreSummary:
    count=item[1][4]
    identify_veterans[f"{item[0]}_favorites"]=item[1][0]/count
    identify_veterans[f"{item[0]}_answers"]=item[1][1]/count
    identify_veterans[f"{item[0]}_score"]=item[1][2]/count
    identify_veterans[f"{item[0]}_views"]=item[1][3]/count


In [23]:
identify_veterans

{'vet_favorites': 1.2930005425935973,
 'vet_answers': 1.2962561041779708,
 'vet_score': 3.5322843190450355,
 'vet_views': 927.7042864894195,
 'brief_favorites': 0.5757105213626953,
 'brief_answers': 0.9704968944099379,
 'brief_score': 2.100084697910785,
 'brief_views': 552.9672971955581}

In [24]:
grader.score('spark_data__identify_veterans', identify_veterans)

Your score: 1.0000


## Question 6: Identify veterans&mdash;full


Same as above, but on the full Stack Exchange data set.

#### Checkpoints

* Total brief users: 1,848,628
* Total veteran users: 288,285

In [4]:
def parseLines(xml_string):
    parser = etree.XMLParser()
    try:
        xml_root = etree.fromstring(xml_string, parser)
    except Exception:
        return False
    return True


good_lines= (sc.textFile('spark-stack-data/allPosts/')
               .filter(lambda txt: len(txt.split())>0 and len(txt.split()[0])>=4 and txt.split()[0][:4]=='<row')
               .filter(parseLines))


In [26]:
user_lines = (sc.textFile('spark-stack-data/allUsers/')
           .filter(lambda txt: len(txt.split())>0 and len(txt.split()[0])>=4 and txt.split()[0][:4]=='<row'))

In [27]:
def FilterUser(xml_string):
    parser = etree.XMLParser()
    try:
        xml_root = etree.fromstring(xml_string, parser)
        Map=xml_root.attrib
        userId=int(Map['Id'])        
    except Exception:
        return False
    return True

user_tuple = namedtuple('user_tuple', ['userId', 'CreationDateUser'])

def parseUser(xml_string):
    parser = etree.XMLParser()
    xml_root = etree.fromstring(xml_string, parser)
    Map=xml_root.attrib
    userId=int(Map['Id'])
    return user_tuple(userId, Map['CreationDate'])

userData= (user_lines.filter(FilterUser)
                     .map(parseUser))


def FilterPost(xml_string):
    parser = etree.XMLParser()
    xml_root = etree.fromstring(xml_string, parser)
    Map=xml_root.attrib
    return 'OwnerUserId' in Map

post_tuple = namedtuple('post_tuple', ['userId','CreationDatePost', 'FavoriteCount', 'postType', 'AnswerCount', 'Score', 'ViewCount'])

def parsePost(xml_string):
    parser = etree.XMLParser()
    xml_root = etree.fromstring(xml_string, parser)
    Map=xml_root.attrib
    userId=int(Map['OwnerUserId'])
    if 'FavoriteCount' not in Map:
        FavoriteCount=0
    else:
        FavoriteCount=int(Map['FavoriteCount'])
    if 'Score' not in Map:
        Score=0
    else:
        Score=int(Map['Score']) 
    if 'ViewCount' not in Map:
        ViewCount=0
    else:
        ViewCount=int(Map['ViewCount'])   
    if 'AnswerCount' not in Map:
        AnswerCount=0
    else:
        AnswerCount=int(Map['AnswerCount'])         
    postType=int(Map['PostTypeId'])    
    return post_tuple(userId, Map['CreationDate'], FavoriteCount, postType, AnswerCount, Score, ViewCount)

postData=(good_lines.filter(FilterPost)
                    .map(parsePost))


In [28]:
def dateDiff(nxt, prev):
    prev_time=datetime.strptime(prev, "%Y-%m-%dT%H:%M:%S.%f")
    nxt_time=datetime.strptime(nxt, "%Y-%m-%dT%H:%M:%S.%f")
    return int(timedelta.total_seconds(nxt_time-prev_time)/86400)

freq_user_type=namedtuple('freq_user_type', ['userId','user_type'])

vetInfo = (postData.map(lambda p: (p.userId, p.CreationDatePost)) 
                        .join(userData.map(lambda u: (u.userId, u.CreationDateUser)))
                        .reduceByKey(lambda a, b: b if (dateDiff(b[0], b[1])>=100 and dateDiff(b[0], b[1])<=150) else a)
                        .map(lambda c: freq_user_type(c[0], 'vet' if (dateDiff(c[1][0], c[1][1])>=100 and dateDiff(c[1][0], c[1][1])<=150) else 'brief')))

In [29]:
FavScoreSummary=(postData.filter(lambda record: record.postType==1)
                         .map(lambda p: (p.userId, (p.FavoriteCount, p.AnswerCount, p.Score, p.ViewCount, p.CreationDatePost)))
                         .join(vetInfo.map(lambda v: (v.userId, v.user_type)))
                         .map(lambda c: ((c[0], c[1][1]), c[1][0]))
                         .reduceByKey(lambda a, b: b if b[4]<a[4] else a)
                         .map(lambda c: (c[0][1], (c[1][0], c[1][1], c[1][2], c[1][3], 1)))
                         .reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1], a[2]+b[2], a[3]+b[3], a[4]+b[4])))

FavScoreSummary=FavScoreSummary.collect() 

identify_veterans_full={}
for item in FavScoreSummary:
    count=item[1][4]
    identify_veterans_full[f"{item[0]}_favorites"]=item[1][0]/count
    identify_veterans_full[f"{item[0]}_answers"]=item[1][1]/count
    identify_veterans_full[f"{item[0]}_score"]=item[1][2]/count
    identify_veterans_full[f"{item[0]}_views"]=item[1][3]/count

In [30]:
identify_veterans_full

{'vet_favorites': 0.8661474978452715,
 'vet_answers': 1.841804785404288,
 'vet_score': 2.2561266751584585,
 'vet_views': 1841.8189951718825,
 'brief_favorites': 0.3854817164205759,
 'brief_answers': 1.5033659787554736,
 'brief_score': 1.129298016272707,
 'brief_views': 1095.1468872554271}

In [31]:
grader.score('spark_data__identify_veterans_full', identify_veterans_full)

Your score: 1.0000


This ends the `spark_data` section.

<br><br><br>

## Spark ML questions

The questions from here forward are associated with the `spark_ml` prefix. They are working with Spark's ML library to do some NLP based analysis on the data.

<br><br><br>

## Question 7: Word2vec


Word2Vec is one approach for vectorizing text data. The vectorized representations of words in the vocabulary tend to be useful for predicting other words in the document, hence the famous example "vector('king') - vector('man') + vector('woman') ~= vector('queen')".

Let's see how good a Word2Vec model we can train using the **tags** of each Stack Exchange post as documents (this uses the full data set). Use the implementation of Word2Vec from Spark ML (this will require using DataFrames) to return a list of the top 25 closest synonyms to "ggplot2" and their similarity score in tuple format ("string", number).

The tags appear in the data as one string, you will need to separate them into individual tags. There is no need to further parse them beyond separating them.

#### Parameters


The dimensionality of the vector space should be 100. The random seed should be 42 in `PySpark`.


#### Checkpoints

* Mean of the top 25 cosine similarities: 0.8012362027168274

In [4]:
from pyspark import SparkContext
from lxml import etree 

sc = SparkContext.getOrCreate()

def parseLines(xml_string):
    parser = etree.XMLParser()
    try:
        xml_root = etree.fromstring(xml_string, parser)
    except Exception:
        return False
    return True

good_lines_small= (sc.textFile('spark-stats-data/allPosts/')
               .filter(lambda txt: len(txt.split())>0 and len(txt.split()[0])>=4 and txt.split()[0][:4]=='<row')
               .filter(parseLines))

good_lines= (sc.textFile('spark-stack-data/allPosts/')
               .filter(lambda txt: len(txt.split())>0 and len(txt.split()[0])>=4 and txt.split()[0][:4]=='<row')
               .filter(parseLines))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [71]:
from collections import namedtuple
import itertools
import numpy as np

tag_tuple = namedtuple('tag_tuple', ['Tags'])

def FilterPostTag(xml_string):
    parser = etree.XMLParser()
    xml_root = etree.fromstring(xml_string, parser)
    Map=xml_root.attrib
    return 'Tags' in Map

def parsePostTag(xml_string):
    parser = etree.XMLParser()
    xml_root = etree.fromstring(xml_string, parser)
    Map=xml_root.attrib
    t=[i.split('>') for i in Map['Tags'].split('<')]
    flat_t=list(itertools.chain(*t))
    filtered_tag=[]
    for item in flat_t:
        if item:
            filtered_tag.append(item.lower())
    return tag_tuple(' '.join(filtered_tag))

postData=(good_lines.filter(FilterPostTag)
                    .map(parsePostTag))

In [72]:
from pyspark.ml.feature import Word2Vec
from pyspark import Row
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

#postDF = postData.toDF(['Tags'])
postDF = postData.map(lambda p: Row(Tags=p.Tags.split())).toDF()

In [73]:
model = Word2Vec(vectorSize=100, inputCol="Tags", outputCol="result", seed=42).fit(postDF)
word2vec=model.findSynonymsArray('ggplot2', 25)
word2vec

[('lattice', 0.889565646648407),
 ('r-grid', 0.8570672869682312),
 ('plotmath', 0.8206846714019775),
 ('line-plot', 0.8155351877212524),
 ('boxplot', 0.8144579529762268),
 ('levelplot', 0.8125491142272949),
 ('plotrix', 0.8074321746826172),
 ('rgl', 0.8073427677154541),
 ('ecdf', 0.8068481683731079),
 ('r-factor', 0.8009346127510071),
 ('gridextra', 0.7989692091941833),
 ('density-plot', 0.7983241081237793),
 ('mgcv', 0.7955276370048523),
 ('categorical-data', 0.7920693755149841),
 ('melt', 0.7909421324729919),
 ('quantile', 0.7893455028533936),
 ('ggvis', 0.7883593440055847),
 ('ggdendro', 0.7859241366386414),
 ('lm', 0.7855767011642456),
 ('gtable', 0.7838269472122192),
 ('standard-error', 0.7834439277648926),
 ('loess', 0.7813327312469482),
 ('hmisc', 0.7783301472663879),
 ('plot', 0.7775034308433533),
 ('geom-text', 0.776115894317627)]

In [74]:
#word2vec = [("data.frame", 0.772650957107544)] * 25

grader.score('spark_ml__word2vec', word2vec)

Your score: 1.0000


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 40892)
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 239, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/

## Question 8: Classification


We'd like to see if we can predict the tags of a **question** from its body text. Instead of predicting specific tags, we will instead try to predict if a question contains one of the top ten most common tags.  

To this end, we have separated out a train and a test set from the original data.  The training and tests sets were downloaded with the stats data at the beginning of the notebook.  You can also get them from S3:
  * `s3://dataincubator-course/spark-stats-data/posts_train.zip`
  * `s3://dataincubator-course/spark-stats-data/posts_test.zip`

This will involve two steps: first, find the ten most common tags for questions in the training data set (the tags have been removed from the test set). Then train a learner to predict from the text of the question (the `Body` attribute) if it should have one of those ten tags in it - you will need to process the question text with NLP techniques such as splitting the text into tokens.

Since we can't reliably pickle Spark models, instead return a list of your predictions, sorted by the question's `Id`.  This sorting is very important, as our grader expects the results to be submitted in a particular order. These predictions should be `0` if the question isn't expected to have a tag in the top ten, and `1` if it is.

As an example, if our top tags include `spark` and `python`, and we had the following questions:

```
<row Body="..." Id="1740" Tags="<machine-learning><spark><regression>" ... />
<row Body="..." Id="723" Tags="<statistics><neurons>" ... />
<row Body="..." Id="2740" Tags="<functional><python><spark><pyspark>" ... />
```

We would expect to return `[0, 1, 1]` (for the order `[723, 1740, 2740]`).  You may need to do some format manipulation in your DataFrame to get this.

#### Checkpoints

- Number of training posts with a tag in the top 10: `22525`
- Number without: `19540`

In [3]:
!unzip -u -d spark-stats-data/train spark-stats-data/posts_train.zip
!unzip -u -d spark-stats-data/test spark-stats-data/posts_test.zip

Archive:  spark-stats-data/posts_train.zip
Archive:  spark-stats-data/posts_test.zip


In [139]:
from pyspark import SparkContext
from lxml import etree 
from collections import namedtuple
import itertools

sc = SparkContext.getOrCreate()

def parseLines(xml_string):
    parser = etree.XMLParser()
    try:
        xml_root = etree.fromstring(xml_string, parser)
    except Exception:
        return False
    return True

training_lines= (sc.textFile('spark-stats-data/train/')
               .filter(lambda txt: len(txt.split())>0 and len(txt.split()[0])>=4 and txt.split()[0][:4]=='<row')
               .filter(parseLines))



post_tuple = namedtuple('tag_tuple', ['Body', 'Tags'])

def FilterPostTag(xml_string):
    parser = etree.XMLParser()
    xml_root = etree.fromstring(xml_string, parser)
    Map=xml_root.attrib
    return 'Tags' in Map and 'Body' in Map and int(Map['PostTypeId'])==1

def parsePostTag(xml_string):
    parser = etree.XMLParser()
    xml_root = etree.fromstring(xml_string, parser)
    Map=xml_root.attrib
    t=[i.split('>') for i in Map['Tags'].split('<')]
    flat_t=list(itertools.chain(*t))
    filtered_tag=[]
    for item in flat_t:
        if item:
            filtered_tag.append(item.lower())
    return post_tuple(Map['Body'], filtered_tag)

trainingPosts=(training_lines.filter(FilterPostTag)
                    .map(parsePostTag))


In [140]:
from pyspark.ml.feature import CountVectorizer
from pyspark import Row
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

postDF = trainingPosts.map(lambda p: Row(Tags=p.Tags)).toDF()
model = CountVectorizer(inputCol="Tags", outputCol="Counts").fit(postDF)
postDF_counts=model.transform(postDF)
                           

In [141]:
import pyspark.sql.functions as f
import heapq

combined_df = (
    postDF_counts.select(f.explode('Tags').alias('col'))
      .select(f.collect_list('col').alias('Tags'))
)

counts = model.transform(combined_df).select('Counts').collect()
vocabMap=dict(zip(model.vocabulary, counts[0]['Counts'].values))

top10=[]
for item in vocabMap:
    count=vocabMap[item]
    if len(top10)<10:
        heapq.heappush(top10, [count, item])
    elif top10[0][0]<count:
        heapq.heappushpop(top10, [count, item])

top10=set([i[1] for i in top10])
top10        

{'correlation',
 'distributions',
 'hypothesis-testing',
 'logistic',
 'machine-learning',
 'probability',
 'r',
 'regression',
 'self-study',
 'time-series'}

In [142]:
tagLabel_tuple = namedtuple('tagLabel_tuple', ['Body', 'label'])

def mapTagLabel(postTuple):
    isThere=False
    for tag in postTuple.Tags:
        if tag in top10:
            isThere=True
            break
    return tagLabel_tuple(postTuple.Body, int(isThere))
        
trainingPostsLabeled=trainingPosts.map(mapTagLabel)

In [143]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

tokenizer = Tokenizer(inputCol='Body', outputCol='words')
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol='features')
logreg = LogisticRegression(maxIter=10, regParam=1.0)

tokens = tokenizer.transform(trainingPostsLabeled.toDF())
hashes = hashingTF.transform(tokens)
model = logreg.fit(hashes)

In [150]:
test_tuple = namedtuple('test_tuple', ['ID', 'Body'])

def filterTest(xml_string):
    parser = etree.XMLParser()
    xml_root = etree.fromstring(xml_string, parser)
    Map=xml_root.attrib
    return int(Map['PostTypeId'])==1

def parseTest(xml_string):
    parser = etree.XMLParser()
    xml_root = etree.fromstring(xml_string, parser)
    Map=xml_root.attrib
    return test_tuple(int(Map['Id']), Map['Body'])


testData=(sc.textFile('spark-stats-data/test/')
               .filter(lambda txt: len(txt.split())>0 and len(txt.split()[0])>=4 and txt.split()[0][:4]=='<row')
               .filter(parseLines)
               .filter(filterTest)         
               .map(parseTest))

In [151]:
testData.count()

4649

In [152]:
test_tokens = tokenizer.transform(testData.toDF())
test_hashes = hashingTF.transform(test_tokens)

predictions = model.transform(test_hashes)
selected = predictions.select('ID', 'prediction')
n=0
for row in selected.take(5):
    print(row)

Row(ID=73939, prediction=1.0)
Row(ID=74047, prediction=0.0)
Row(ID=74050, prediction=0.0)
Row(ID=74054, prediction=1.0)
Row(ID=74066, prediction=1.0)


In [169]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[tokenizer, hashingTF, logreg])
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

paramGrid = (ParamGridBuilder() 
    .addGrid(hashingTF.numFeatures, [1000, 2000, 3000, 4000, 5000, 6000, 7000]) 
    .addGrid(logreg.regParam, [10, 1, .1, 0.01, 0.001]) 
    .build())

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=10,
                          seed=42)

cvModel = crossval.fit(trainingPostsLabeled.toDF())

better_predictions = cvModel.transform(testData.toDF())

In [170]:
#better_predictions.select('ID', 'prediction')
final_pred=better_predictions.select('ID', 'prediction').collect()

In [171]:
classification=[i[1] for i in sorted([[row['ID'], int(row['prediction'])] for row in final_pred])]

In [172]:
len(classification)

4649

In [173]:
#classification = [0] * 4649

grader.score('spark_ml__classification', classification)

Your score: 0.9189


## K-means (ungraded)


From your trained Word2Vec model, pass the vectors into a K-means clustering algorithm. Create a plot of the sum of squared error by calculating the square root of the sum of the squared distances for each point and its assigned cluster. For an independent variable use either the number of clusters k or the dimension of the Word2Vec vectorization.

*Copyright &copy; 2023 Pragmatic Institute. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*